# Extracting Stock Data from Yahoo Finance

The first step before forecasting stock prices, we need to extract prices for all stocks.<br>
To do this we need to know which stocks (tickers) are in each S&P 500 and S&P 400.

In [26]:
import pandas as pd
import numpy as np
import datetime as dt
import pandas_datareader.data as web

import bs4 as bs
#import pickle
import requests
from tqdm import tqdm_notebook as tqdm

The following function web-scrapes using the bs4 - Beautiful package to get all ticker data available in a tablular form on Wikipedia.

In [27]:
def save_sp_tickers(link, c, t):
    resp = requests.get(link) #get webpage from link and store it in a variable
    soup = bs.BeautifulSoup(resp.text, 'lxml') # convert into lxml to extract data that is needed
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    comp_name = []
    for row in table.findAll('tr')[1:]:
        company = row.findAll('td')[c].text # Use column to get Company Name
        ticker = row.findAll('td')[t].text # Use column to get Ticker
        comp_name.append(company)
        tickers.append(ticker)

    return comp_name, tickers

# Function that creates a Data
def make_df(**kwargs): 
    return pd.DataFrame(kwargs) 

# Yahoo uses '-' instead of '.' in its ticker symbols 
def edit_ticker(col): # to edit according to Yahoo's rules
    for i in range(len(col)):
        col[i] = col[i].replace('.','-')

In [28]:
#function call to Get ticker and company name 
sp400_c, sp400_t = save_sp_tickers('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies', 0, 1)
sp500_c, sp500_t = save_sp_tickers('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies', 1, 0)

# store it in a DataFrame
sp400_info = make_df(Name=sp400_c, Ticker=sp400_t)
sp500_info = make_df(Name=sp500_c, Ticker=sp500_t)

# Remove extra'\n' to tickers
sp400_info.Name = sp400_info.Name.map(lambda x: x.rstrip('\n'))
sp500_info.Ticker = sp500_info.Ticker.map(lambda x: x.rstrip('\n'))

# function call to make tickers compatible with yahoo
edit_ticker(sp500_info.Ticker)
edit_ticker(sp400_info.Ticker)

In [54]:
start = dt.datetime(1970, 1, 1) # Using 1970 as the base year
end = dt.datetime(2019, 9, 29) # End Date is till Sept 2019

# Function to get prices to a ticker from start date to end date
def get_price(col): 
    df = pd.DataFrame()
    for stock in tqdm(list(col)):
        #print(stock, end = '')
        df[stock] = web.DataReader(stock, 'yahoo', start, end)['Adj Close']
    
    return df

#Get all 900 Companies Price data
sp500_df = get_price(sp500_info.Ticker)

In [50]:
sp400_info = sp400_info.drop(155, axis=0)

In [53]:
sp400_df = get_price(sp400_info.Ticker)

Once data has been downloaded, I store it in a csv so that I can use it from my local machine and I dont have to get it from the web everytime. 

In [56]:
sp500_df.to_csv('data/sp500.csv', encoding='utf-8')
sp400_df.to_csv('data/sp400.csv', encoding='utf-8')

This stores the Company Information that is - Company Name & Ticker for our reference if needed

In [57]:
sp500_info.to_csv('data/sp500_info.csv', encoding='utf-8', index=False)
sp400_info.to_csv('data/sp400_info.csv', encoding='utf-8', index=False)